# Практика
Используемые библиотеки

In [1]:
import pandas as pd
import numpy as np
import requests # for web-download
import io # for web-download
import re # for data processing


## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [4]:
#url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
#s = requests.get(url).content
#df=pd.read_csv(io.StringIO(s.decode('UTF8')))
### Если не работает загрузка on-line
df=pd.read_csv("Химический анализ родника в Нескучном саду.csv", sep=';')
df

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,б/цвета,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


In [108]:
# Ваше решение
import re


index = 0
df['Норматив'] = df['Норматив'].str.replace(',', '.')
for iterator in df['Норматив']:
    
    standard = re.findall(r'[0-9.]+', df.loc[index, 'Норматив'])
    result = df.loc[index, 'Результат анализа']
    
    standard = [float(i) for i in standard]
    standard.sort()
    
    if str(iterator).lower().__contains__('в пределах'):
        if len(standard) == 2:
            try:
                result = float(result)
                if  (result >=  standard[0]) & (result <= standard[1]):
                    df.loc[index, 'Интерпретация'] = 'Норма'
                elif result < standard[0]:
                    df.loc[index, 'Интерпретация'] = 'Ниже нормы'
                elif result > standard[1]:
                    df.loc[index, 'Интерпретация'] = 'Выше нормы'
            except ValueError:
                df.loc[index, 'Интерпретация'] = 'Ошибка в результате'
     
    elif str(iterator).lower().__contains__('не более'):           
        try:
                result = float(result)
                if result <= standard[len(standard) - 1]:
                    df.loc[index, 'Интерпретация'] = 'Норма'
                elif result > standard[len(standard) - 1]:
                    df.loc[index, 'Интерпретация'] = 'Выше нормы'
        except ValueError:
                df.loc[index, 'Интерпретация'] = 'Ошибка в результате'      
    index +=1

df.set_index('Показатель')

,Единица измерений,Результат анализа,Норматив,Интерпретация
Показатель,,,,
pH,единицы pH,8.4,в пределах 6-9,Норма
Запах,баллы,1,не более 2-3,Норма
Цветность,градусы,б/цвета,не более 30,Ошибка в результате
Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10,Норма
Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,не более 1.5,Норма
Нитриты (по NO2),мг/дм3,0.017,не более 3.3,Норма
Нитраты (по NO3),мг/дм3,24,не более 45,Норма
Фосфаты (P),мг/дм3,0.36,не более 3.5,Норма
Хлориды (Cl),мг/дм3,200,не более 350,Норма


## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?


In [148]:
# Ваше решение
import random
box = ['orange']*5 + ['apple']*4

n = 100000

A = pd.DataFrame({"A":list(map(lambda a: random.choice([random.choice(box)]),  range(n))),
                  "B":list(map(lambda a: random.choice([random.choice(box)]),  range(n))),
                 "C":list(map(lambda a: random.choice([random.choice(box)]),  range(n)))
                 })
# считаем статистику выпадений апелсьинов три раза подряд
P_orange = len(A[(A.A=='orange') & (A.B=='orange') & (A.C=='orange')])/A.shape[0]

P_orange
#Вероятность немного отличается на практике в большую сторону

0.1714

### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [ ]:
# Ваше решение

In [197]:
box = ['stand']*3 + ['non-s']*7
n = 100000
A = pd.DataFrame({"1st":list(map(lambda a: random.choice([random.choice(box)]),  range(n))),
                  "2nd":list(map(lambda a: random.choice([random.choice(box)]),  range(n))),
                 "3rd":list(map(lambda a: random.choice([random.choice(box)]),  range(n))),
                  "4th":list(map(lambda a: random.choice([random.choice(box)]),  range(n))),
                  "5th":list(map(lambda a: random.choice([random.choice(box)]),  range(n))),
                  "6th":list(map(lambda a: random.choice([random.choice(box)]),  range(n))),
                  "7th":list(map(lambda a: random.choice([random.choice(box)]),  range(n))),
                  "8th":list(map(lambda a: random.choice([random.choice(box)]),  range(n))),
                  "9th":list(map(lambda a: random.choice([random.choice(box)]),  range(n))),
                  "10th":list(map(lambda a: random.choice([random.choice(box)]),  range(n))),
                 })


#Вероятность, что наткнется на стандартную деталь со 2-й проверки
P_stand = len(A[(A['2nd'] == 'stand')])/A.shape[0]
P_stand
               
#Вероятность на практике немного отличается в большую сторону

0.29772